## Do polar plots of all records in selected day

In [ ]:
using CSV, CairoMakie
using Dates, DataFrames, Distributions, DSP
#using Gtk
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics #, StatsPlots
#using Tk


function do_subplot(x, y, z)
############################    
    ax = subplot(x, y, z, polar="true") 
    ax.grid(:true, fontsize=:10)
    ax.set_rlabel_position(-90)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    
    PyPlot.title(string(z))

end    # do_subplot()
    

function smooth_spectra(Pden_in, sample_frequency)
##################################################
# smooth the spectra into bands centered on 0.05Hz spacing (i.e. 0:0.005:0.64)
    nyquist = sample_frequency/2

    freq_in = range(0, stop=nyquist, length=length(Pden_in))

    freq_out = [0.0]
    Pden_smoothed = [mean(Pden_in[1:8])]

    i = 9
    while i <= length(Pden_in)

        push!(freq_out,freq_in[i+8])

        if i < length(Pden_in)-16

            push!(Pden_smoothed, mean(Pden_in[i:i+16]))

        end

        i+=16

    end

    push!(Pden_smoothed, mean(Pden_in[end-8:end]))
            
    return (freq_out, Pden_smoothed)
        
end


function get_Fourier_coefficients(heave, north, west)
#####################################################    
    # Get the cross periodograms
    cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);
    cps_north_north = mt_cross_power_spectra([north north]', fs=sample_frequency);
    cps_west_west = mt_cross_power_spectra([west west]', fs=sample_frequency);

    cps_north_heave = mt_cross_power_spectra([north heave]', fs=sample_frequency);
    cps_west_heave = mt_cross_power_spectra([west heave]', fs=sample_frequency);
    cps_north_west = mt_cross_power_spectra([north west]', fs=sample_frequency);

##    fhh = cps_heave_heave.freq
    fhh, Chh = smooth_spectra(real.(cps_heave_heave.power[1,1,:]), sample_frequency)

    #fnn = cps_north_north.freq
    fhh, Cnn = smooth_spectra(real.(cps_north_north.power[1,1,:]), sample_frequency)

    #fww = cps_west_west.freq
    fhh, Cww = smooth_spectra(real.(cps_west_west.power[1,1,:]), sample_frequency)

    #fnw = cps_north_west.freq
    fhh, Cnw = smooth_spectra(real.(cps_north_west.power[1,2,:]), sample_frequency)

    #fnh = cps_north_heave.freq
    fhh, Qnh = smooth_spectra(imag.(cps_north_heave.power[1,2,:]), sample_frequency)

    #fwh = cps_west_heave.freq
    fhh, Qwh = smooth_spectra(imag.(cps_west_heave.power[1,2,:]), sample_frequency)

    a1 = Qnh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5
    b1 = -Qwh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5

    a2 = (Cnn .- Cww) ./ (Cnn .+ Cww)
    b2 = -2 .* Cnw ./ (Cnn .+ Cww)
    
    return(fhh, Chh, a1, b1, a2, b2)
    
end    # get_Fourier_coefficients()


function get_displacements(arry)
#####################################
    
    displacements = []

    if length(arry[1]) == 3
    
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
        end
        
    else
        
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
        end
        
    end

    displacements[findall(>=(2048), displacements)] = 2048 .- displacements[findall(>=(2048), displacements)];
    
    return(displacements./100)
    
end     # get_displacements()


function get_HNW(infil)
#####################################
        
    global df = DataFrame(CSV.File(infil,header=0, delim=",", types=String));

    # Calculate sequence numbers
    arry = SubString.(df.Column1, 3, 4)

    global sequence = []

    for i in arry
        append!(sequence,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
    end

    arry = SubString.(df.Column3, 1, 3);
    heave = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column3, 4, ) .* SubString.(df.Column4, 1, 2)
    north = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column4, 3, 4) .* SubString.(df.Column5, 1, 1)
    west = get_displacements(arry);

    return(heave, north, west)

    end    # get_HNW()


function plot_polar(fig, displacement_df, row, col, total, spec_max)
####################################################################
    
    Chh = displacement_df.Chh[total]
    a1 = displacement_df.a1[total]
    b1 = displacement_df.b1[total] 
    a2 = displacement_df.a2[total] 
    b2 = displacement_df.b2[total]
    time_string  = displacement_df.Time_string[total]

#==
    aa = length(Chh)

    r = 1:6:aa
    ρ = r ./ (aa/nyquist) 

    θ = 0:pi/180:2pi

    mat =  []

    for j in r

        for i in θ

            push!(mat,Chh[j] * (a1[j]*cos(i) + b1[j]*sin(i) + a2[j]*cos(2i) + b2[j]*sin(2i)))

        end

    end

    mat[mat .< 0] .= 0
            
    mat = reshape(mat, length(θ), length(r))
==#
    aa = length(Chh) # Number of spectral points

    r = 1:6:aa
    global ρ = r ./ (aa/nyquist) 

    global θ = 0:pi/180:2pi        

    # populate a matrix of spectral surface values
    mat = [Chh[r1] * (a1[r1]*cos.(θ) + b1[r1]*sin.(θ) + a2[r1]*cos.(2θ) + b2[r1]*sin.(2θ)) for r1 in r]

    mat = hvcat(size(mat,1),mat...)

    # set any values less than zero to zero
    mat[mat .< 0] .= 0

    time_string = displacement_df.Time_string[total]

    cmap = Reverse(:ocean)
    levels = round(spec_max/100, digits=2):round(spec_max/20, digits=2):round(spec_max, digits=2)

    ax = CairoMakie.PolarAxis(fig[row, col],
    thetaticklabelsize = 15,  
    rlimits=(0,0.4), rticklabelsize=15, rticks=0:0.2:0.4, rgridwidth=0.5, rtickangle=180, rminorgridvisible=true, rminorgridstyle=:dot,
    theta_0=-pi/2, thetagridwidth=0.5, thetaminorgridvisible=true, thetaminorgridstyle=:dot, thetaminorticks=IntervalsBetween(3), 
    direction=-1, width=330, height=310, title=time_string, titlesize=18,
    )

    CairoMakie.contourf!(ax, θ, ρ, Float64.(mat), colormap=cmap, levels=levels) # 0.02:0.05:1,)
    CairoMakie.contour!(ax, θ, ρ, Float64.(mat), colormap=cmap, levels=levels)
    
    return(fig)
    
    end    # plot_polar()


#######################################################################################
#######################################################################################
#######################################################################################    

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
const sample_frequency = 1.28

hxv_directory = pick_folder()

# build list of all hxv files in selected directory
hxv_files = filter(x->occursin(".hxv",x), readdir(hxv_directory));
hxv_files = hxv_files[findall(x->endswith(uppercase(x), ".HXV"), hxv_files)];

nyquist = sample_frequency/2

# build df containing displacements and Fourier coefficient for selected day
displacement_df = DataFrame(Time_string = [], Heave = [], North = [], West = [], fhh = [], Chh = [], a1 = [], b1 = [], a2 = [], b2 = [])

println("Reading files:")
flush(stdout)

fl_num = 1
for fl in hxv_files
    
    infil = hxv_directory * "\\" * fl
    date_string = split(hxv_directory,"\\")[end]
    global time_string = replace(split(infil,date_string)[end][2:6], 'h' => ':')

    if (mod(fl_num,10) == 0)
        print(string(fl_num))
    else
        print(".")
    end
        
    global heave, north, west = get_HNW(infil)
    fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)
    
    push!(displacement_df, (time_string, heave, north, west, fhh, Chh, a1, b1, a2, b2))
    fl_num += 1
end
    
#######################################################################################
#######################################################################################
#######################################################################################    

fig = CairoMakie.Figure(size=(1200, 2500))

date_string = split(hxv_directory,"\\")[end]
println("\nSpectral peak for "*date_string*" occurred at "*displacement_df.Time_string[argmax(maximum.(displacement_df.Chh))])
    
supertitle = fig[0, :] = Label(fig, date_string,
fontsize = 24, color = (:grey, 0.25))

# get the highest energy value for the day
spec_max = maximum(maximum.(displacement_df.Chh))

total=0
println("\nPreparing plots now:")
for row = 1:8

    for col in 1:6

        total+=1        
        
        if (mod(total,10) == 0)
            print(string(total))
        else
            print(".")
        end
        
        try
            
            fig = plot_polar(fig, displacement_df, row, col, total, spec_max)
        
        catch
            
            println("Alert: Not all 48 records available for this day")
            break
            
        end    
    
    end
    
end

CairoMakie.Colorbar(fig[9, :], limits=(0, round(spec_max, digits=1, RoundUp)), label="Spectral Density (m²/Hz.)", labelsize=:20, 
        width=500, height=30, vertical=false, flipaxis=false, colormap=Reverse(:ocean))

resize_to_layout!(fig)

fig
##==
try
    site_string = split(hxv_directory, "\\")[end-2]
##    CairoMakie.save(site_string*"_"*date_string*"_polar_plots.png", fig, px_per_unit = 1)
            CairoMakie.save(".\\"*split(hxv_directory, "\\")[end-1]*"_"*split(hxv_directory, "\\")[end]*"_polar_plots.png", fig, px_per_unit = 1)
catch
    "Alert: Plot not saved!"
end
    
#==#

display(fig)

In [ ]:
CairoMakie.save("Mool_"*date_string*"_polar_plots.png", fig, px_per_unit = 1)

In [ ]:
# 3d animation of Heave, North, and West WSEs

using GLMakie

fig = GLMakie.Figure()
ax = GLMakie.Axis3(fig[1, 1])
Makie.lines!(north, west, heave, 
    markersize=5, linewidth=0.25,
    color=heave, colormap=Reverse(:diverging_gwr_55_95_c38_n256))

Makie.scatter!(north, west, heave, 
    markersize=5, linewidth=0.25,
    color=heave, colormap=Reverse(:diverging_gwr_55_95_c38_n256))

start_angle = π / 4
n_frames = 600
ax.viewmode = :fit # Prevent axis from resizing during animation
record(fig, "test.mp4", 1:n_frames, framerate = 30) do frame
    ax.azimuth[] = start_angle + 2pi * frame / n_frames
end

## Animation of polar plot of spectra and direction

In [ ]:
using CSV, CairoMakie
using Dates, DataFrames, Distributions, DSP
using GLMakie
using LaTeXStrings
using NativeFileDialog
#using Plots, Printf, PyPlot
#using Statistics #, StatsPlots
#using Tk


function do_subplot(x, y, z)
############################    
    ax = subplot(x, y, z, polar="true") 
    ax.grid(:true, fontsize=:10)
    ax.set_rlabel_position(-90)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    
    PyPlot.title(string(z))

end    # do_subplot()
    

function smooth_spectra(Pden_in, sample_frequency)
##################################################
# smooth the spectra into bands centered on 0.05Hz spacing (i.e. 0:0.005:0.64)
    nyquist = sample_frequency/2

    freq_in = range(0, stop=nyquist, length=length(Pden_in))

    freq_out = [0.0]
    Pden_smoothed = [mean(Pden_in[1:8])]

    i = 9
    while i <= length(Pden_in)

        push!(freq_out,freq_in[i+8])

        if i < length(Pden_in)-16

            push!(Pden_smoothed, mean(Pden_in[i:i+16]))

        end

        i+=16

    end

    push!(Pden_smoothed, mean(Pden_in[end-8:end]))
            
    return (freq_out, Pden_smoothed)
        
end    # smooth_spectra()


function get_Fourier_coefficients(heave, north, west)
#####################################################    
    # Get the cross periodograms
    cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);
    cps_north_north = mt_cross_power_spectra([north north]', fs=sample_frequency);
    cps_west_west = mt_cross_power_spectra([west west]', fs=sample_frequency);

    cps_north_heave = mt_cross_power_spectra([north heave]', fs=sample_frequency);
    cps_west_heave = mt_cross_power_spectra([west heave]', fs=sample_frequency);
    cps_north_west = mt_cross_power_spectra([north west]', fs=sample_frequency);

##    fhh = cps_heave_heave.freq
    fhh, Chh = smooth_spectra(real.(cps_heave_heave.power[1,1,:]), sample_frequency)

    #fnn = cps_north_north.freq
    fhh, Cnn = smooth_spectra(real.(cps_north_north.power[1,1,:]), sample_frequency)

    #fww = cps_west_west.freq
    fhh, Cww = smooth_spectra(real.(cps_west_west.power[1,1,:]), sample_frequency)

    #fnw = cps_north_west.freq
    fhh, Cnw = smooth_spectra(real.(cps_north_west.power[1,2,:]), sample_frequency)

    #fnh = cps_north_heave.freq
    fhh, Qnh = smooth_spectra(imag.(cps_north_heave.power[1,2,:]), sample_frequency)

    #fwh = cps_west_heave.freq
    fhh, Qwh = smooth_spectra(imag.(cps_west_heave.power[1,2,:]), sample_frequency)

    a1 = Qnh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5
    b1 = -Qwh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5

    a2 = (Cnn .- Cww) ./ (Cnn .+ Cww)
    b2 = -2 .* Cnw ./ (Cnn .+ Cww)
    
    return(fhh, Chh, a1, b1, a2, b2)
    
end    # get_Fourier_coefficients()


function get_displacements(arry)
#####################################
    
    displacements = []

    if length(arry[1]) == 3
    
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
        end
        
    else
        
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
        end
        
    end

    displacements[findall(>=(2048), displacements)] = 2048 .- displacements[findall(>=(2048), displacements)];
    
    return(displacements./100)
    
end     # get_displacements()


function get_HNW(infil)
#####################################
        
    global df = DataFrame(CSV.File(infil,header=0, delim=",", types=String));

    # Calculate sequence numbers
    arry = SubString.(df.Column1, 3, 4)

    global sequence = []

    for i in arry
        append!(sequence,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
    end

    arry = SubString.(df.Column3, 1, 3);
    heave = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column3, 4, ) .* SubString.(df.Column4, 1, 2)
    north = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column4, 3, 4) .* SubString.(df.Column5, 1, 1)
    west = get_displacements(arry);

    return(heave, north, west)

    end    # get_HNW()


function get_spec_dir(displacement_df, total)

    Chh = displacement_df.Chh[total]
    a1 = displacement_df.a1[total]
    b1 = displacement_df.b1[total] 
    a2 = displacement_df.a2[total] 
    b2 = displacement_df.b2[total]
    time_string[] = displacement_df.Time_string[total]

    aa = length(Chh) # Number of spectral points

    r = 1:3:aa
    ρ = r ./ (aa/nyquist) 

    θ = 0:pi/180:2pi

#    mat =  []

    mat = [Chh[j] * (a1[j]*cos(i) + b1[j]*sin(i) + a2[j]*cos(2i) + b2[j]*sin(2i)) for i in θ, j in r]

    mat[mat .< 0] .= 0
    
    return(θ, ρ, mat, time_string)
    
end    # get_spec_dir()


# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
const sample_frequency = 1.28

hxv_directory = pick_folder()

# build list of all hxv files in selected directory
hxv_files = filter(x->occursin(".hxv",x), readdir(hxv_directory));
hxv_files = hxv_files[findall(x->endswith(uppercase(x), ".HXV"), hxv_files)];

nyquist = sample_frequency/2

# build df containing displacements and Fourier coefficient for selected day
displacement_df = DataFrame(Time_string = [], Heave = [], North = [], West = [], fhh = [], Chh = [], a1 = [], b1 = [], a2 = [], b2 = [], mat = [])

println("Processing each 30-minute record for selected day")
flush(stdout)

fl_num = 1
for fl in hxv_files
    
    infil = hxv_directory * "\\" * fl
    date_string = split(hxv_directory,"\\")[end]
    global time_string = replace(split(infil,date_string)[end][2:6], 'h' => ':')

    if (mod(fl_num,10) == 0)
        print(string(fl_num))
    else
        print(".")
    end
        
    global heave, north, west = get_HNW(infil)
    fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)
 
    aa = length(Chh) # Number of spectral points

    r = 1:6:aa
    global ρ = r ./ (aa/nyquist) 

    global θ = 0:pi/180:2pi        

    # populate a matrix of spectral surface values
    mat = [Chh[r1] * (a1[r1]*cos.(θ) + b1[r1]*sin.(θ) + a2[r1]*cos.(2θ) + b2[r1]*sin.(2θ)) for r1 in r]

    mat = hvcat(size(mat,1),mat...)

    # set any values less than zero to zero
    mat[mat .< 0] .= 0

    # add spectral data to plot df
    push!(displacement_df, (time_string, heave, north, west, fhh, Chh, a1, b1, a2, b2, mat));
    fl_num += 1
end
    
#######################################################################################
#######################################################################################
#######################################################################################    

fig = CairoMakie.Figure(size=(1200, 3000))

date_string = split(hxv_directory,"\\")[end]
println("\nSpectral peak for "*date_string*" occurred at "*displacement_df.Time_string[argmax(maximum.(displacement_df.Chh))])

println("\nPlotting 30-minute records now")
flush(stdout)

# get the highest energy value for the day
# this sets scaling of plots                    
spec_max = maximum(maximum.(displacement_df.mat))
                    
println("Maximum spectra for day = ",round(spec_max, digits=2),"m²/Hz.")
                    
# declare the Observables
inc = Observable(1)
time_string = Observable(displacement_df[inc[],:].Time_string)
mat = Observable(Float64.(displacement_df[inc[],:].mat))
fhh_L = Observable(Int(round(length(displacement_df.Chh[1]) / 6)))
        
fig = CairoMakie.Figure(size=(800, 850))
Label(fig[0, :], text = date_string, fontsize = 25)

# draw the polar axis
ax = CairoMakie.PolarAxis(fig[1, 1],
    thetaticklabelsize = 15,  
    rlimits=(0,0.4), rticklabelsize=15, rticks=0:0.2:0.4, rgridwidth=0.5, rtickangle=180, rminorgridvisible=true, rminorgridstyle=:dot,
    theta_0=-pi/2, thetagridwidth=0.5, thetaminorgridvisible=true, thetaminorgridstyle=:dot, thetaminorticks=IntervalsBetween(3), 
    direction=-1, width=630, height=610, title=time_string, titlesize=24,
    )

# Set plotting values
cmap = Reverse(:ocean)
levels = round(spec_max/100, digits=2):round(spec_max/20, digits=2):round(spec_max, digits=2)
θ = 0:pi/180:2pi
ρ = range(0, stop=0.64, length=fhh_L[])

# do contour plot
c1 = CairoMakie.contourf!(ax, θ, ρ, mat, colormap=cmap, levels=levels)
c1 = CairoMakie.contour!(ax, θ, ρ, mat, colormap=cmap, levels=levels)
                    
ax = CairoMakie.Colorbar(fig[2, 1], limits=(0, round(spec_max, digits=1, RoundUp)), label="Spectral Density (m²/Hz.)", labelsize=:20, 
            width=500, height=30, vertical=false, flipaxis=false, colormap=cmap)  
display(fig) 

# update the Observables
for i in 1:nrow(displacement_df)
    
    inc[] = i
    time_string[] = displacement_df[inc[],:].Time_string
    
    try
        mat[] = Float64.(displacement_df.mat[inc[]])
    catch
        break
    end
    
    sleep(0.3)
    yield()
    
    
end

In [ ]:
frames = 1:nrow(displacement_df)

site_string = split(hxv_directory, "\\")[end-2]
##file_name = site_string*"_"*date_string*"_polar_plots.mp4"
file_name = ".\\"*split(hxv_directory, "\\")[end-1]*"_"*split(hxv_directory, "\\")[end]*"_polar_plots.mp4"

record(fig, file_name, frames;
        framerate = 2) do frame
    
#    inc = frame
    time_string[] = displacement_df[frame,:].Time_string
    mat[] = Float64.(displacement_df.mat[frame])
                    
    if (mod(frame,10) == 0)
        print(string(frame))
    else
        print(".")
    end
                  
end

In [ ]:
site_string = split(hxv_directory, "\\")[end-2]
site_string*"_"*date_string*"_polar_plots.mp4"

In [ ]:
file_name